In [1]:
import pandas as pd


In [7]:
df=pd.read_csv('/home/harshit/csv files/sales_prediction.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [8]:
X=df.drop(columns=['Item_Outlet_Sales'])
Y=df['Item_Outlet_Sales']
X.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1


In [9]:
X.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [10]:
mean=df['Item_Weight'].mean()
mean

np.float64(12.857645184135976)

In [11]:
X_c=X.copy()
X_c.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [12]:
df[['Item_Type','Item_Weight']]

,Item_Type,Item_Weight
0,Dairy,9.300
1,Soft Drinks,5.920
2,Meat,17.500
3,Fruits and Vegetables,19.200
4,Household,8.930
...,...,...
8518,Snack Foods,6.865
8519,Baking Goods,8.380
8520,Health and Hygiene,10.600
8521,Snack Foods,7.210


In [13]:
Pivot_weight=X_c.pivot_table(index='Item_Type',values='Item_Weight',aggfunc='mean').reset_index()
WeightType_map=dict(zip(Pivot_weight['Item_Type'],Pivot_weight['Item_Weight']))
WeightType_map

{'Baking Goods': 12.277108208955223,
 'Breads': 11.346936274509805,
 'Breakfast': 12.768202247191013,
 'Canned': 12.305705009276437,
 'Dairy': 13.42606890459364,
 'Frozen Foods': 12.867061281337046,
 'Fruits and Vegetables': 13.224769381746812,
 'Hard Drinks': 11.400327868852461,
 'Health and Hygiene': 13.14231395348837,
 'Household': 13.384736495388669,
 'Meat': 12.81734421364985,
 'Others': 13.853284671532847,
 'Seafood': 12.552843137254904,
 'Snack Foods': 12.987879554655871,
 'Soft Drinks': 11.847459893048129,
 'Starchy Foods': 13.69073076923077}

In [14]:
def insert_weights(data_frame):
    data_frame.loc[:,'Item_Weight']=data_frame.loc[:,'Item_Weight'].fillna(data_frame.loc[:,'Item_Type'].map(WeightType_map))
    return data_frame


In [15]:
X_c=insert_weights(X_c)
X_c.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [16]:
X_c.groupby(by=['Outlet_Size','Outlet_Location_Type']).size()

Outlet_Size  Outlet_Location_Type
High         Tier 3                   932
Medium       Tier 1                   930
             Tier 3                  1863
Small        Tier 1                  1458
             Tier 2                   930
dtype: int64

In [17]:
X_c.groupby(by=['Outlet_Size','Outlet_Type']).size()

Outlet_Size  Outlet_Type      
High         Supermarket Type1     932
Medium       Supermarket Type1     930
             Supermarket Type2     928
             Supermarket Type3     935
Small        Grocery Store         528
             Supermarket Type1    1860
dtype: int64

In [18]:
OutletSize_Mapping = dict(X_c.groupby('Outlet_Type')['Outlet_Size'].agg(lambda x: x.mode()[0]))
OutletSize_Mapping

{'Grocery Store': 'Small',
 'Supermarket Type1': 'Small',
 'Supermarket Type2': 'Medium',
 'Supermarket Type3': 'Medium'}

In [19]:
def insert_size(data_frame):
    data_frame.loc[:,'Outlet_Size']=data_frame.loc[:,'Outlet_Size'].fillna(data_frame.loc[:,'Outlet_Type'].map(OutletSize_Mapping))
    return data_frame

In [20]:
X_c=insert_size(X_c)
X_c.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [21]:
X_c['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [22]:
def fat_content(data_frame):
    data_frame['Item_Fat_Content']=data_frame['Item_Fat_Content'].replace({
        'LF':'Low Fat',        
        'reg': 'Regular',         
        'low fat' : 'Low Fat'    
    })
    return data_frame
X_c=fat_content(X_c)

In [23]:
X_c['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [24]:
X_c['Item_Identifier'].str[:2].value_counts()

Item_Identifier
FD    6125
NC    1599
DR     799
Name: count, dtype: int64

In [25]:
def gen_itemType(data_frame):
    data_frame['ItemType_gen']=data_frame['Item_Identifier'].str[:2]
    mapp={
        'FD':'Food',
        'NC':'Non-consumable',
        'DR':'Drink'
    }
    data_frame.replace(mapp,inplace=True)
    return data_frame
X_c=gen_itemType(X_c)
X_c.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,ItemType_gen
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,Food
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,Drink
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,Food
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,Food
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,Non-consumable


In [26]:
X_c[['ItemType_gen','Item_Fat_Content']].value_counts()

ItemType_gen    Item_Fat_Content
Food            Low Fat             3190
                Regular             2935
Non-consumable  Low Fat             1599
Drink           Low Fat              728
                Regular               71
Name: count, dtype: int64

In [27]:
def item_fat_content(data_frame):
    data_frame.loc[data_frame['ItemType_gen']=='Non-Consumable','Item_Fat_Content']='Non_Edible'
    return data_frame
X_c=item_fat_content(X_c)
X_c[['ItemType_gen','Item_Fat_Content']].value_counts()


ItemType_gen    Item_Fat_Content
Food            Low Fat             3190
                Regular             2935
Non-consumable  Low Fat             1599
Drink           Low Fat              728
                Regular               71
Name: count, dtype: int64

In [28]:
def prepare_data(data_frame):
    data_frame=insert_weights(data_frame)
    data_frame=insert_size(data_frame)
    data_frame=fat_content(data_frame)
    data_frame=gen_itemType(data_frame)
    data_frame=item_fat_content(data_frame)
    return data_frame


In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X=X_c

In [31]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
X_train_c=X_train.copy()

In [32]:
X_train_c.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,ItemType_gen
549,FDW44,9.500,Regular,0.035206,Fruits and Vegetables,171.3448,OUT049,1999,Medium,Tier 1,Supermarket Type1,Food
7757,NCF54,18.000,Low Fat,0.047473,Household,170.5422,OUT045,2002,Small,Tier 2,Supermarket Type1,Non-consumable
764,FDY03,17.600,Regular,0.076122,Meat,111.7202,OUT046,1997,Small,Tier 1,Supermarket Type1,Food
6867,FDQ20,8.325,Low Fat,0.029845,Fruits and Vegetables,41.6138,OUT045,2002,Small,Tier 2,Supermarket Type1,Food
2716,FDP34,12.850,Low Fat,0.137228,Snack Foods,155.5630,OUT046,1997,Small,Tier 1,Supermarket Type1,Food


In [33]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [34]:
X_train_c.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
ItemType_gen                  object
dtype: object

In [35]:
le_columns=X_train_c[['Outlet_Size','Outlet_Location_Type']]
le.fit(le_columns['Outlet_Size'])
le.fit(le_columns['Outlet_Location_Type'])

LabelEncoder()

In [36]:
for column in X_train_c[['Outlet_Size','Outlet_Location_Type']]:
    X_train_c.loc[:,column]=le.fit_transform(X_train_c.loc[:,column])
X_train_c

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,ItemType_gen
549,FDW44,9.500,Regular,0.035206,Fruits and Vegetables,171.3448,OUT049,1999,1,0,Supermarket Type1,Food
7757,NCF54,18.000,Low Fat,0.047473,Household,170.5422,OUT045,2002,2,1,Supermarket Type1,Non-consumable
764,FDY03,17.600,Regular,0.076122,Meat,111.7202,OUT046,1997,2,0,Supermarket Type1,Food
6867,FDQ20,8.325,Low Fat,0.029845,Fruits and Vegetables,41.6138,OUT045,2002,2,1,Supermarket Type1,Food
2716,FDP34,12.850,Low Fat,0.137228,Snack Foods,155.5630,OUT046,1997,2,0,Supermarket Type1,Food
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,FDY08,9.395,Regular,0.286345,Fruits and Vegetables,139.1838,OUT010,1998,2,2,Grocery Store,Food
5191,FDC41,15.600,Low Fat,0.117575,Frozen Foods,75.6670,OUT017,2007,2,1,Supermarket Type1,Food
5390,NCQ53,17.600,Low Fat,0.018944,Health and Hygiene,237.3590,OUT045,2002,2,1,Supermarket Type1,Non-consumable
860,FDL46,20.350,Low Fat,0.054363,Snack Foods,117.9466,OUT017,2007,2,1,Supermarket Type1,Food


In [37]:
cat_feats=X_train_c.select_dtypes(include='object').drop(columns=['Item_Identifier','Outlet_Identifier','Item_Type'])
num_feats=X_train.select_dtypes(exclude='object').reset_index()

In [38]:
ohe.fit(cat_feats)

OneHotEncoder()

In [39]:
ohe_features_names=ohe.get_feature_names_out(input_features=cat_feats.columns)
cat_feats_ohe=pd.DataFrame(ohe.transform(cat_feats).toarray(),columns=ohe_features_names)
cat_feats_ohe

,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,ItemType_gen_Drink,ItemType_gen_Food,ItemType_gen_Non-consumable
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6813,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6814,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6815,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
6816,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [40]:
X_train_final=pd.concat([cat_feats_ohe,num_feats],axis=1)
X_train_final.shape,X_test.shape

((6818, 20), (1705, 12))

In [41]:
le_columns=X_test[['Outlet_Size','Outlet_Location_Type']]
le.fit(le_columns['Outlet_Size'])
le.fit(le_columns['Outlet_Location_Type'])
for column in X_test[['Outlet_Size','Outlet_Location_Type']]:
    X_test.loc[:,column]=le.fit_transform(X_test.loc[:,column])
    
cat_feats_test=X_test.select_dtypes(include='object').drop(columns=['Item_Identifier','Outlet_Identifier','Item_Type'])
num_feats_test=X_test.select_dtypes(exclude='object').reset_index()
ohe.fit(cat_feats_test)
ohe_features_names=ohe.get_feature_names_out(input_features=cat_feats_test.columns)
cat_feats_ohe_test=pd.DataFrame(ohe.transform(cat_feats_test).toarray(),columns=ohe_features_names)

In [47]:
X_test.isnull().any().any()

np.False_

In [48]:
X_test_final=pd.concat([cat_feats_ohe_test,num_feats_test],axis=1)
X_train_final=X_train_final.drop(columns='index')
X_test_final=X_test_final.drop(columns='index')

X_train_final.shape,X_test_final.shape

((6818, 19), (1705, 19))

In [49]:
from sklearn.ensemble import RandomForestRegressor,HistGradientBoostingRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [50]:
def modelling(model,X_train,Y_train,X_test,Y_test,cv=10):
    cv_results=cross_validate(model,X_train,Y_train,cv=cv)
    model.fit(X_train,Y_train)
    y_pred=model.predict(X_test) 
    print('Model:', model)
    print("R²:", r2_score(Y_test,y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(Y_test,y_pred)))

In [51]:
from sklearn.model_selection import RandomizedSearchCV
def hyperparameter(model,parameters,cv):
    classifier=RandomizedSearchCV(model,parameters,cv=cv)
    classifier.fit(X_train_final,Y_train)
    print(model,classifier.best_params_)


In [52]:
params={
    'rf':{
    'n_estimators':[80,100,150],
    'max_depth':[3,5,10],
    'min_samples_leaf':[1,5,10]
},
'hgb':{
    'max_iter':[80,100,150,300],
    'max_depth':[3,5,7,10,50],
    'learning_rate':[0.1,0.3,0.5,0.8,0.9],
    'min_samples_leaf':[10,20,30,50]
},
'gbr':{
    'n_estimators':[80,100,150,300],
    'max_depth':[3,5,7,10,50],
    'learning_rate':[0.1,0.3,0.5,0.8,0.9],
    'min_samples_leaf':[1,5,10]
}
}
model_map = {
    'rf': RandomForestRegressor(),
    'gbr': GradientBoostingRegressor(),
    'hgb': HistGradientBoostingRegressor()
}

In [53]:
for key,model in model_map.items():
    hyperparameter(model,params[key],cv=5)

RandomForestRegressor() {'n_estimators': 150, 'min_samples_leaf': 10, 'max_depth': 5}
GradientBoostingRegressor() {'n_estimators': 300, 'min_samples_leaf': 5, 'max_depth': 3, 'learning_rate': 0.1}
HistGradientBoostingRegressor() {'min_samples_leaf': 50, 'max_iter': 100, 'max_depth': 5, 'learning_rate': 0.1}


In [67]:
rf=RandomForestRegressor(n_estimators= 150,max_depth=5, min_samples_leaf=10,random_state=42)
modelling(rf, X_train_final, Y_train,X_test_final,Y_test)

Model: RandomForestRegressor(max_depth=5, min_samples_leaf=10, n_estimators=150,
                      random_state=42)
R²: 0.6183459704796923
RMSE: 1018.49115265664


In [55]:
hgr=HistGradientBoostingRegressor(min_samples_leaf= 20, max_iter= 100,max_depth= 3,learning_rate= 0.1,random_state=42)
modelling(hgr,X_train_final, Y_train,X_test_final,Y_test)

Model: HistGradientBoostingRegressor(max_depth=3, random_state=42)
R²: 0.6139340754965674
RMSE: 1024.3610821097416


In [68]:
gbr=GradientBoostingRegressor(n_estimators= 300, min_samples_leaf=5, max_depth=3,learning_rate= 0.1,random_state=42)
modelling(gbr,X_train_final, Y_train,X_test_final,Y_test)

Model: GradientBoostingRegressor(min_samples_leaf=5, n_estimators=300, random_state=42)
R²: 0.6025231069267559
RMSE: 1039.3893868733185


In [70]:
import platform
import sys
from datetime import datetime
import joblib

In [71]:
with open("Sales Forecast Prediction.ipynb", "r") as f:
    source_code = f.read()

In [72]:
import sklearn
metadata = {
    "timestamp": str(datetime.now()),
    "model": RandomForestRegressor(),
    "training_data_shape": X_train_final.shape,
    "r2_score": r2_score,
    "sklearn_version": sklearn.__version__,
    "python_version": sys.version,
    "platform": platform.platform(),
    "source_code":source_code
}
joblib.dump(metadata, "model.pkl")

['model.pkl']